# Calculate the nonstationary internal tide component

# Dask worker setup instructions

https://github.com/mrayson/pawsey-containers/tree/master/hpc-python/jupyter-sfoda

These settings do the job:

 1. `dask-scheduler --scheduler-file scheduler-$HOSTNAME.json --idle-timeout 0`
 2. `ssh localhost "cd /software/projects/pawsey0106/mrayson/projects && sbatch --ntasks=16 -c 16 start_worker.slm scheduler-$HOSTNAME.json ./"`


In [1]:
hostname = !echo $(hostname)
hostname[0]

'nid002793'

In [2]:
import os
from distributed import Client
client=Client(scheduler_file='../../scheduler-' + hostname[0] + '.json')
client

Connection method: Scheduler file,Scheduler file: ../../scheduler-nid002793.json
Dashboard: http://10.253.137.81:8787/status,
Comm: tcp://10.253.137.81:8786,Workers: 0
Dashboard: http://10.253.137.81:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
client.restart() # This clears all data

Connection method: Scheduler file,Scheduler file: ../../scheduler-nid002793.json
Dashboard: http://10.253.137.81:8787/status,
Comm: tcp://10.253.137.81:8786,Workers: 0
Dashboard: http://10.253.137.81:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
import sfoda.suntans.sunxarray as sxr

import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from cmocean import cm
from s3fs import S3FileSystem, S3Map

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-8k1vnx24 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
s3login = '/home/mrayson/code/s3.acacia.login'

with open(s3login) as f:
    lines = f.readlines()

url, key, secret = [ff.strip('\n') for ff in lines[0:3]]
s3 = S3FileSystem(client_kwargs={'endpoint_url':url},
            key=key,
            secret=secret)


In [8]:
## Process the steric height
# f = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_ssh_steric.zarr/", s3=s3)
# varname = 'eta_steric'
# longname = 'steric sea surface height'
# dropvars=['eta_steric']

## Process the raw sea surface height
f = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_surface.zarr/", s3=s3)
varname = 'eta'
longname = 'sea surface height'
dropvars=['uc','vc','temp','eta']

cutoff_dt = 40*3600
cutoff_dt_high = 4*3600
dtsec = 3600.

# Output file
zarrfile = 'NWS_2km_GLORYS_2013_2014_ssh_filtered.zarr'
# zarrfile = 'NWS_2km_GLORYS_2013_2014_ssh_steric_filtered_v2.zarr'

In [9]:
ds = xr.open_dataset(f, engine='zarr', chunks={})

# These need filling in??
ds.suntans.xv=ds.xv.values
ds.suntans.yv=ds.yv.values

ds

Changing the _FillValue from -999999 to 999999


<xarray.Dataset>
Dimensions:  (Nc: 225368, numsides: 8, Nk: 80, time: 8772, Np: 454016)
Coordinates:
  * time     (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-07-01
    z_r      (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
Dimensions without coordinates: Nc, numsides, Nk, Np
Data variables:
    cells    (Nc, numsides) int32 dask.array<chunksize=(2000, 8), meta=np.ndarray>
    dv       (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    dz       (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
    eta      (time, Nc) float64 dask.array<chunksize=(745, 2000), meta=np.ndarray>
    nfaces   (Nc) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
    temp     (time, Nc) float64 dask.array<chunksize=(745, 2000), meta=np.ndarray>
    uc       (time, Nc) float64 dask.array<chunksize=(745, 2000), meta=np.ndarray>
    vc       (time, Nc) float64 dask.array<chunksize=(745, 2000), meta=np.ndarray>
    xp       (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    xv       (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    yp       (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    yv       (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
Attributes:
    Author:       mrayson
    Description:  SUNTANS surface fields

In [10]:
# Need to fix the time because the restart at each month lead to non-continuous time stepping
# Find and remove duplicate time entries
tsec = (ds.time.values-ds.time.values[0]).astype(float)/1e9
dt = np.ones(tsec.shape)

dt[1:] = tsec[1:]-tsec[0:-1]
dt[0] = dt[1]*1

#plt.plot(dt)

#badidx = dt<dt.max()
#dt[np.argwhere(badidx)]

remove_idx = dt==60.
# fix_idx = dt==3540.

# myds.time.isel(time=fix_idx) + np.timedelta64(60,'s')
ds2 = ds.isel(time=~remove_idx).chunk({'time':-1})


In [11]:
ds2[varname]


<xarray.DataArray 'eta' (time: 8761, Nc: 225368)>
dask.array<rechunk-merge, shape=(8761, 225368), dtype=float64, chunksize=(8761, 2000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-07-01
Dimensions without coordinates: Nc
Attributes:
    location:   face
    long_name:  Sea surface elevation
    mesh:       suntans_mesh
    units:      m

In [12]:
from scipy import signal


def filt(ytmp, cutoff_dt, dt, btype='low', order=8, ftype='sos', axis=-1):
    """
    Butterworth filter the time series

    Inputs:
        cutoff_dt - cuttoff period [seconds]
        btype - 'low' or 'high' or 'band'
    """
    if not btype == 'band':
        Wn = dt/cutoff_dt
    else:
        Wn = [dt/co for co in cutoff_dt]
    
    if ftype=='sos':
        sos = signal.butter(order, Wn, btype, analog=False, output='sos')
        return signal.sosfiltfilt(sos, ytmp, axis=axis)
    else:
        (b, a) = signal.butter(order, Wn, btype=btype, analog=0, output='ba')
        return signal.filtfilt(b, a, ytmp, axis=axis)

def testfunc(arr, axis=0):
    return arr*2

In [13]:
%%time
#from functools import partial
arr = ds2[varname]

# Compute the filted variable
# arr_filt = xr.apply_ufunc(testfunc,
#                           arr, 
#                           dask='parallelized',
#                           output_dtypes=[arr.dtype],
#                           input_core_dims=([],),
#                           #output_core_dims=(['time','Nc']),
#                           kwargs={'axis': 0},
#                          )

# # Compute the filted variable
arr_lowpass = xr.apply_ufunc(filt,
                          arr, 
                          cutoff_dt,
                          dtsec,
                          dask='parallelized',
                          output_dtypes=[arr.dtype],
                          #input_core_dims=(['time','Nc'], [], []),
                          #output_core_dims=(['time','Nc']),
                          kwargs={'axis': 0},
                          #dask_gufunc_kwargs={'allow_rechunk':True}
                         )


arr_lowpass

CPU times: user 6.89 ms, sys: 0 ns, total: 6.89 ms
Wall time: 6.89 ms


<xarray.DataArray 'eta' (time: 8761, Nc: 225368)>
dask.array<transpose, shape=(8761, 225368), dtype=float64, chunksize=(8761, 2000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-07-01
Dimensions without coordinates: Nc

In [14]:

arr_highpass = xr.apply_ufunc(filt,
                          ds2[varname], 
                          #[cutoff_dt, cutoff_dt_high],
                          cutoff_dt_high,
                          dtsec,
                          dask='parallelized',
                          output_dtypes=[arr.dtype],
                          #input_core_dims=(['time','Nc'], [], []),
                          #output_core_dims=(['time','Nc']),
                          kwargs={'axis': 0, 'btype':'high'},
                          #dask_gufunc_kwargs={'allow_rechunk':True}
                         )

arr_highpass

<xarray.DataArray 'eta' (time: 8761, Nc: 225368)>
dask.array<transpose, shape=(8761, 225368), dtype=float64, chunksize=(8761, 2000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-07-01
Dimensions without coordinates: Nc

In [15]:
arr_bandpass = arr - arr_lowpass - arr_highpass

In [16]:
# Create an output data set

dsout = ds2.drop_vars(dropvars)

dsout['{}_lowpass'.format(varname)] = xr.DataArray(arr_lowpass, dims=('time','Nc'), 
                                           attrs=
                                {'long_name':longname,'units':'m','location':'face',
                                'filter_cutoff_sec':cutoff_dt} )

dsout['{}_bandpass'.format(varname)] = xr.DataArray(arr_bandpass, dims=('time','Nc'), 
                                           attrs=
                                {'long_name':longname,'units':'m','location':'face',
                                'filter_cutoff_sec':cutoff_dt_high} )
dsout

<xarray.Dataset>
Dimensions:       (Nc: 225368, numsides: 8, Nk: 80, time: 8761, Np: 454016)
Coordinates:
  * time          (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-07-01
    z_r           (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
Dimensions without coordinates: Nc, numsides, Nk, Np
Data variables:
    cells         (Nc, numsides) int32 dask.array<chunksize=(2000, 8), meta=np.ndarray>
    dv            (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    dz            (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
    nfaces        (Nc) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
    xp            (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    xv            (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    yp            (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    yv            (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    eta_lowpass   (time, Nc) float64 dask.array<chunksize=(8761, 2000), meta=np.ndarray>
    eta_bandpass  (time, Nc) float64 dask.array<chunksize=(8761, 2000), meta=np.ndarray>
Attributes:
    Author:       mrayson
    Description:  SUNTANS surface fields

In [17]:
outfile = '/scratch/pawsey0106/mrayson/SUNTANS_NWS_2km/'+zarrfile

dsout.to_zarr(outfile, mode='w', append_dim=None, compute=True)
outfile

'/scratch/pawsey0106/mrayson/SUNTANS_NWS_2km/NWS_2km_GLORYS_2013_2014_ssh_filtered.zarr'

In [18]:
# Save time chunked output
d = S3Map("uwaoceanprocesses-suntans/"+zarrfile, s3=s3)

dsout.to_zarr(store=d, mode='w', compute=True, safe_chunks=False)

In [19]:
zarrfile

'NWS_2km_GLORYS_2013_2014_ssh_filtered.zarr'